In [ ]:
from typing import Optional, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda'

block_size = 8
batch_size = 32
max_iterations = 100000
learning_rate = 1e-4
eval_iters = 250
dropout = 0.2

embedding_size = 512
num_heads = 16
num_encoder_layers = 4

In [ ]:
with open('dataset/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocabulary_size = len(chars)

In [ ]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}
encode_function = lambda s: [string_to_int[c] for c in s]
decode_function = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode_function(text), dtype=torch.long)

In [ ]:
train_portion = 0.8
cutoff_index = int(train_portion * len(data))
train_data = data[:cutoff_index]
val_data = data[cutoff_index:]


def get_batch(split: str) -> Union[torch.Tensor, torch.Tensor]:
    data = train_data if split != 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, embedding_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_size, embedding_size * 4),
            nn.ReLU(),
            nn.Linear(embedding_size * 4, embedding_size),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = nn.MultiHeadAttention(embedding_size, num_heads)
        self.ffwd = FeedForward(embedding_size)
        self.norm1 = nn.LayerNorm(embedding_size)
        self.norm2 = nn.LayerNorm(embedding_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.sa(x)
        x = x + y
        x = self.norm1(x)
        y = self.ffwd(x)
        x = x + y
        x = self.norm2(x)
        
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embedding_table = nn.Embedding(block_size, embedding_size)
        self.blocks = nn.Sequential([*[Block(embedding_size, num_heads) for i in range(num_encoder_layers)]])
        self.layer_norm = nn.LayerNorm(embedding_size)
        self.linear_pool = nn.Linear(embedding_size, vocab_size)

        self.apply(self.__init_weights)

    def __init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index: torch.Tensor, targets: Optional[torch.Tensor] = None) -> Union[torch.Tensor, torch.Tensor]:
        token_embed = self.token_embedding_table(index)
        B, T, C = token_embed.shape
        position_embed = self.position_embedding_table(torch.arange(T, device=device))
        x = token_embed + position_embed
        x = self.blocks(x)
        x = self.layer_norm(x)
        logits = self.linear_pool(x)
        

        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index: torch.Tensor, max_new_tokens: int) -> int:
        for i in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index


model = GPTLanguageModel(vocabulary_size)
m = model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iterations):
    x_batch, y_batch = get_batch('train')
    logits, loss = model(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % eval_iters == 0:
        losses = estimate_loss()
        print(f"Iteration {i}, train loss {losses['train']}, val loss {losses['val']}")

print(loss.item())